<a href="https://colab.research.google.com/github/gabrie1-s/phosphorus_analysis/blob/main/Analise_fosforo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Importação das bibliotecas e leitura dos dados

In [ ]:
%%capture
!pip install Bayesian-Optimization
!pip install xgboost
!pip install ray

In [ ]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
import random
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
import os
import gc
import pdb
import ray
import keras
import xgboost
import tensorflow
from scipy import stats
from sklearn import metrics
from xgboost import XGBRegressor
from keras.models import load_model
from bayes_opt import BayesianOptimization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
drive.mount('/content/drive/')

In [ ]:
pentecoste = pd.read_excel("/content/drive/MyDrive/CNN_Chagas/fosforo/Analise_fosforo.xlsx", sheet_name="Pentecoste")
acarape = pd.read_excel("/content/drive/MyDrive/CNN_Chagas/fosforo/Analise_fosforo.xlsx", sheet_name="Acarape do meio")
aracoiaba = pd.read_excel("/content/drive/MyDrive/CNN_Chagas/fosforo/Analise_fosforo.xlsx", sheet_name="Aracoiaba")

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
def prepare_data(df):
  df.columns = df.iloc[0]
  df = df.iloc[1:, :]
  df = df[["P medido (mg/L)","TOA  B2", "TOA B3"]]
  df.dropna(inplace=True)
  return df

In [ ]:
pentecoste = prepare_data(pentecoste.copy())
pentecoste = pentecoste.iloc[:, 1:]

acarape = prepare_data(acarape.copy())
aracoiaba = prepare_data(aracoiaba.copy())

#2. Análise exploratória e preprocessamento

In [ ]:
dataset = pd.concat([pentecoste, acarape, aracoiaba])
dataset = dataset.astype(float)

In [ ]:
dataset['B3/B2'] = dataset['TOA B3'] / dataset['TOA  B2']

In [ ]:
dataset.columns

##2.1. Breve observação

In [ ]:
sns.pairplot(dataset, corner=True)
plt.show()

Observe no conjunto de gráficos de dispersão acima que a variável B2 parece estar altamente correlacionada com a variável B3. Notamos isso pelo comportamento quase linear dos pontos do gráfico 'TOA B2' x 'TOA B3'. Em contrapartida, não parece haver relção entre B3 e 'P medido' ou B2 e 'P medido'.  Além disso, no mapa de calor fica evidente a maior correlação da variável P com a razão B3/B2. Embora tal correlação não seja significativa, é consideravelmente maior que as demais correlações da variável P.

In [ ]:
#Sem transformação
ax = sns.heatmap(dataset.corr(), annot=True, vmin=-1, vmax=1)
ax.set_title("Mapa de calor da correlação entre as variáveis")

No mapa de calor acima, podemos visualizar as correlações entre as variáveis. Vemos que as informações do mapa de calor apenas confirmam o que foi dito anteriormente: as variáveis B2 e B3 são altamente correlacionadas entre si e ambas tem pouca (ou nenhuma) correlação com a variável P. Além disso, no mapa de calor fica evidente a maior correlação da variável P com a razão B3/B2. Embora tal correlação não seja significativa, é consideravelmente maior que as demais correlações da variável P.

$\textbf{Nota}$ : a correlação entre duas variáveis $x$ e $y$ é dada por:
\begin{equation}
  \rho_{x, y} = \frac{covariance(x, y)}{\sigma_{x} \cdot \sigma_{y}}
\end{equation}

O valor de $\rho_{x,y}$ é tal que: $-1 \leq \rho_{x,y} \leq 1$. Quanto maior o módulo de $\rho_{x,y}$ maior a correlação entre $x$ e $y$. Quanto mais próximo $\rho_{x,y}$ for de zero, menor a correlação.

Como o módulo da correla B2 e B3, significa que a variável B2 acresecnta muito pouca informação a B3 e vice-versa. Em casos de correlação tão elevada, normalmente, elimina-se uma das variáveis ou substitui-se as variáveis por uma combinação linear de ambas. Entretanto, como desconheço o problema (só me foi dito que deveria tentar predizer P com base, apenas, em B2 e B3 😅) decidi manter ambas as variáveis.

In [ ]:
b2 = np.array(dataset[dataset.columns[1]])
b3 = np.array(dataset[dataset.columns[2]])

b2 = (b2-b2.min())/((b2.max()-b2.min()))
b3 = (b3-b3.min())/((b3.max()-b3.min()))

b3 = [x for _, x in sorted(zip(b2, b3))]
b2 = sorted(b2)

plt.rcParams["figure.figsize"] = (20,6)

plt.scatter(range(len(b2)), b2)
plt.scatter(range(len(b2)), b3)
plt.title('B2 x B3 comparison')
plt.ylabel('Value')
plt.xlabel('Samples')
plt.legend(['B2', 'B3'], loc='upper left')
plt.show()

Como forma de visualizar melhor essa correlação, fizmeos o plot acima, perceba que os valores de B3 seguem a mesma tendência de crescimento de B2.

**OBS**.: Para plotarmos o gráfico acima, ordenamos os valores de B2 e reposicionamos os valores de B3 de forma a não alterar os pares (B2, B3) do conjunto de dados original.

##2.2. Análise da skewness (obliquidade)

In [ ]:
dataset.hist(
    figsize=(20, 8),
    grid = False,
    rwidth = 0.8,
    bins = 100
)
plt.show()

In [ ]:
dataset.skew()

Geralmente, skewnesses (obliquidades) maiores que 1 ou menores que -1 são consideradas elevadas. Testaremos dois métodos para reduzir o valor da skewness.

###2.2.1. Método do Box-cox

Essa trasnformação é dad por:

\begin{equation}
      y(\lambda) =
      \begin{cases}
          \frac{y^\lambda - 1}{\lambda} & \text{se $\lambda \neq 0$}\\
          log(y) & \text{se $\lambda = 0$}
      \end{cases}
\end{equation}

A ideia é encontrar um valor de $\lambda$ (um escalar) que mais aproxima a distribuição $y(\lambda)$ de uma distribuição normal

In [ ]:
test_data = dataset.copy()
lbs = []

for col in test_data.columns[:-1]:
  test_data[col], l = stats.boxcox(test_data[col])[0:2]
  lbs.append(l)
test_data.hist(
    figsize=(20, 8),
    grid = False,
    rwidth = 0.8,
    bins = 100
)
plt.show()

In [ ]:
test_data.skew()

In [ ]:
test_data.columns

In [ ]:
lbs

Cada valor na lista acima refere-se ao valor de $\lambda$ de cada variável: 'P medido', 'TOA  B2', 'TOA B3', respectivamente.

###2.2.2. Método de Yeo-Johnson

Essa trasnformação é dad por:

\begin{equation}
      \psi(y, \lambda) =
      \begin{cases}
          \frac{(y+1)^\lambda - 1}{\lambda} & \text{se $y \geq 0 \ e \ \lambda \neq 0$}\\
          log(y+1) & \text{se $y \geq 0 \ e \ \lambda = 0$}\\
          -\frac{(-y+1)^{2-\lambda} - 1}{2 - \lambda} & \text{se $y < 0 \ e \ \lambda \neq 2$}\\
          -log(-y+1) & \text{se $y < 2 \ e \ \lambda = 2$}\\
      \end{cases}
\end{equation}

In [ ]:
test_data = dataset.copy()
lbs = []

for col in test_data.columns[:-1]:
  test_data[col], l = stats.yeojohnson(test_data[col])[0:2]
  lbs.append(l)

test_data.hist(
    figsize=(20, 8),
    grid = False,
    rwidth = 0.8,
    bins = 100
)
plt.show()

In [ ]:
test_data.skew()

In [ ]:
print(test_data.columns)
print(lbs)

Cada valor na lista acima refere-se ao valor de $\lambda$ de cada variável: 'P medido', 'TOA  B2', 'TOA B3', respectivamente.

Os resultados foram semelhantes, mas, no caso do Box-cox, a redução da skewness da variável 'P medido' foi bem mais significativa, logo, optaremos por este método.

Note que, quando aplicarmos essa transformação, as unidades das variáveis também serão alteradas.

In [ ]:
for col in dataset.columns[:-1]:
  dataset[col] = stats.boxcox(dataset[col])[0]

##2.3. Análise de outliers

In [ ]:
plt.rcParams["figure.figsize"] = (20,6)
fig, axs = plt.subplots(1, 4)

i=0
for col in dataset:
  axs[i].set_title(col)
  axs[i].boxplot(dataset[col])
  i +=1

plt.show()

Após o tratamento da skewness quase não observamos a presença de outliers. Devido a isso, optaremos por mantê-los.

In [ ]:
'''def plot_boxplot(df, ft):
  df.boxplot(column = [ft])
  plt.grid(False)
  plt.show()

def outliers(df, ft):
  Q1 = df[ft].quantile(0.25)
  Q3 = df[ft].quantile(0.75)
  I = Q3 - Q1
  LwLimit = Q1 - 1.5*I
  UpLimit = Q3 + 1.5*I
  ls = df.index[(df[ft] < LwLimit) | (df[ft] > UpLimit)]
  return ls

def remove(df, ls):
  ls = sorted(set(ls))
  df = df.drop(ls)
  return df

def apply_remotion(dataset):
  remover = []
  for i in dataset:
    remover.extend(outliers(dataset, i))

  dataset_cleaned = remove(dataset, remover)
  return dataset_cleaned

dataset = apply_remotion(dataset)'''

In [ ]:
'''plt.rcParams["figure.figsize"] = (20,6)
fig, axs = plt.subplots(1, 3)

i=0
for col in dataset:
  axs[i].set_title(col)
  axs[i].boxplot(dataset[col])
  i +=1

plt.show()'''

Aqui, decidimos manter os outliers, visto que eles são poucos e temos poucos dados

##2.4. Análise bivariada / correlações entre os dados

In [ ]:
sns.pairplot(dataset)
plt.show()

fig, axes = plt.subplots(2, 2, figsize=(20, 10), sharey=True, constrained_layout=True)
fig.suptitle('Comparação entre os métodos')

#Spearman
ax = sns.heatmap(dataset.corr(method='spearman'), annot=True, vmin=-1, vmax=1, ax = axes[0,0])
axes[0,0].set_title("Spearman")
bottom, top = ax.get_ylim()

#Pearson
ax = sns.heatmap(dataset.corr(method='pearson'), annot=True, vmin=-1, vmax=1, ax = axes[0,1])
axes[0,1].set_title("Pearson")
bottom, top = ax.get_ylim()

#Kendall
ax = sns.heatmap(dataset.corr(method='kendall'), annot=True, vmin=-1, vmax=1, ax = axes[1,0])
bottom, top = ax.get_ylim()
axes[1,0].set_title("Kendall")

#Sem transformação
ax = sns.heatmap(dataset.corr(), annot=True, vmin=-1, vmax=1, ax = axes[1,1])
axes[1,1].set_title("Sem transformação")
bottom, top = ax.get_ylim()

Aqui a análise é semelhante à que fizemos no início: B2 e B3 são altamente correlacionados e, mesmo usando métodos não lineares de correlação (Pearson, Kendall e Spearman) a correlação deles com P é baixa. Além disso, a correlação de spearman da variável B3/B2 com a variável P é moderada. Isso nos leva às seguintes conclusões.



*   A informação das variáveis B2 e B3 é, em grande parte, reduntante (como já vimos anteriormente);
*   Como não existe nenhuma correlação forte com P, dificilmente métodos lineares de regressão terão bom desempenho ao tentar predizer P a partir de B2, B3 e B3/B2.



In [ ]:
b2 = np.array(dataset[dataset.columns[1]])
b3 = np.array(dataset[dataset.columns[2]])

b2 = (b2-b2.min())/((b2.max()-b2.min()))
b3 = (b3-b3.min())/((b3.max()-b3.min()))

b3 = [x for _, x in sorted(zip(b2, b3))]
b2 = sorted(b2)

plt.scatter(range(len(b2)), b2)
plt.scatter(range(len(b2)), b3)
plt.title('B2 x B3 comparison')
plt.ylabel('Value')
plt.xlabel('Samples')
plt.legend(['B2', 'B3'], loc='upper left')
plt.show()

Aqui, fizemos esse plot apenas para mostrar que B2 e B3 continuam altamente correlacionadas, mesmo após as transformações.

#3. Algoritmos de regressão

Dividiremos o conjunto de dados em treino e teste. Deixamos o conjunto de teste com apenas 15% das amostras, pois, como temos poucos dados, queremos maximizar o número de amostras de treinamento. Por esse mesmo motivo, optamos por usar cross validation ao invés de criar um conjunto de validação.

In [ ]:
x_tr, x_te, y_tr, y_te = train_test_split(dataset[dataset.columns[1:]], dataset[dataset.columns[0]], test_size=0.15, random_state=3)

In [ ]:
def plot_results(y_pred, y_tes):
  plt.scatter(range(len(y_pred)), y_pred, c='r')
  plt.plot(range(len(y_tes)), y_tes, linestyle="-", marker="o", label="Expenses")
  plt.title('Model performance - test set')
  plt.ylabel('P medido')
  plt.xlabel('Sample')
  plt.legend(['predicted', 'real'], loc='upper left')
  plt.show()

Antes de implementarmos os modelos de regressão, gostaria de deixar claro que usaremos duas métricas principais para avaliar o desempenho de nossos modelos:



*   Erro percentual médio absoluto (mean absolute percentage error):
\begin{equation}
  MAPE = \frac{1}{n} \sum_{i = 1}^{n} \frac{|y_i - p_i|}{y_i}
\end{equation}

*   Erro médio absoluto (mean absolute error):
\begin{equation}
  MAE = \frac{1}{n} \sum_{i = 1}^{n} |y_i - p_i|
\end{equation}

*   $R^2$ score:
\begin{equation}
  R^2 = NSE = 1 - \frac{\sum_{i=1}^{N} (y_i - p_i)^2}{\sum_{i=1}^{N} (y_i - \bar{y_i})^2}
\end{equation}

Onde, $y_i$ é o valor real,  $p_i$ é o valor predito pelo modelo testado e $\bar{y_i}$ é a média dos valores de $y_i$, $\forall i$.


##3.1. Apenas ignore esta parte

In [ ]:
def normalization(data, apply_log=False, scale=True, exponential=False):
  df = data.drop('P medido (mg/L)', axis=1)

  if apply_log:
    df = df.astype(float)
    df = np.log(df)

  if exponential:
    df = df.astype(float)
    df.applymap(np.exp)

  if scale:
    for col in df.columns:
      df[col] = df[col]/df[col].max()

    targ = 'P medido (mg/L)'
    data[targ] = data[targ]/data[targ].max()

  df_norm = pd.concat((df, data['P medido (mg/L)']), 1)

  return df_norm

In [ ]:
'''pentecoste = normalization(pentecoste.copy(), apply_log=False, exponential=True, scale=False)
acarape = normalization(acarape.copy(), apply_log=False, exponential=True, scale=False)
aracoiaba = normalization(aracoiaba.copy(), apply_log=False, exponential=True, scale=False)'''

In [ ]:
'''x_tr, y_tr = np.array(pentecoste[pentecoste.columns[0:2]]), np.array(pentecoste["P medido (mg/L)"])
x_va, y_va = np.array(acarape[acarape.columns[0:2]]), np.array(acarape["P medido (mg/L)"])
x_te, y_te = np.array(aracoiaba[aracoiaba.columns[0:2]]), np.array(aracoiaba["P medido (mg/L)"])

x_tr = np.concatenate([x_tr, x_va])
y_tr = np.concatenate([y_tr, y_va])
del x_va, y_va

x_tr = x_tr.astype('float64')
y_tr = y_tr.astype('float64')
x_te = x_te.astype('float64')
y_te = y_te.astype('float64')'''

##3.2. Métodos Lineares

In [ ]:
import sklearn.model_selection as skms
import sklearn.linear_model as sklm
import sklearn as sk
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score, mean_absolute_percentage_error
from sklearn.metrics import make_scorer

In [ ]:
def train_test_model(model, x_tr, y_tr, x_te, y_te, folds=5):
  kf = KFold(n_splits = folds, random_state = 1, shuffle=True)
  scores = cross_validate(model, x_tr, y_tr, cv=kf,
                        scoring = ('neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'r2'))

  mae = np.mean(scores['test_neg_mean_absolute_error']*(-1))
  std_mae = np.std(scores['test_neg_mean_absolute_error']*(-1))

  mape = np.mean(scores['test_neg_mean_absolute_percentage_error']*(-1))
  std_mpae = np.std(scores['test_neg_mean_absolute_percentage_error']*(-1))

  r2 = np.mean(scores['test_r2'])
  std_r2 = np.std(scores['test_r2'])

  print('--------------------Validação Cruazada-----------------------')
  print("Média dos valore de MAE: " + str(mae))
  print("Desvio padrão dos valore de MAE: " + str(std_mae) + "\n")

  print("Média dos valore de MAPE: " + str(mape))
  print("Desvio padrão dos valore de MAPE: " + str(std_mpae) + "\n")

  print("Média dos valore de R2: " + str(r2))
  print("Desvio padrão dos valore de R2: " + str(std_r2) + "\n")

  print('--------------------Teste-----------------------')
  y_pred = model.predict(x_te)
  print("MAE: " + str(mean_absolute_error(y_te, y_pred)))
  print("MAPE: " + str(mean_absolute_percentage_error(y_te, y_pred)))
  print("R2: " + str(r2_score(y_te, y_pred)))

  plot_results(model.predict(x_te), y_te)

###3.2.1. Regressão Linear

In [ ]:
linear_regressor = sklm.LinearRegression()
linear_regressor.fit(x_tr, y_tr)

**Teste com 5-Folds**

In [ ]:
train_test_model(linear_regressor, x_tr, y_tr, x_te, y_te, folds=5)

**Teste com 10-Folds**

In [ ]:
train_test_model(linear_regressor, x_tr, y_tr, x_te, y_te, folds=10)

In [ ]:
print("Coeficientes da regressão linear: " + str(linear_regressor.coef_))

###3.2.2. Rgressão Ridge

In [ ]:
ray.init()

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
# definindo espaço de busca para os lambdas
lambdas = np.linspace(0, 100, 1000)

**Teste com 5-Folds**

In [ ]:
@ray.remote

def test_lambda(lamb):
  # aplicando a regressão
  ridge = sklm.Ridge(alpha = lamb)
  ridge.fit(x_tr, y_tr)             # Fit a ridge regression on the training data

  scores = skms.cross_validate(ridge, x_tr, y_tr, cv=5,
                               scoring=('r2', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error'))

  mae = np.mean(scores['test_neg_mean_absolute_error']*(-1))
  mape = np.mean(scores['test_neg_mean_absolute_percentage_error']*(-1))
  r2 = np.mean(scores['test_r2'])

  return [mae, mape, r2]

In [ ]:
result_values = ray.get([test_lambda.remote(i) for i in lambdas])
scores_MAE = [result[0] for result in result_values]
scores_MAPE = [result[1] for result in result_values]
scores_R2 = [result[2] for result in result_values]

In [ ]:
best_index = scores_MAE.index(min(scores_MAE))

print("Lambda:" + str(lambdas[best_index]))
print("Best MAE: " + str(scores_MAE[best_index]))
print("Best MAPE: " + str(scores_MAPE[best_index]))
print("R2: " + str(scores_R2[best_index]))

ridge = sklm.Ridge(alpha = lambdas[best_index])
ridge.fit(x_tr, y_tr)
plot_results(ridge.predict(x_te), y_te)

**Teste com 10-folds**

In [ ]:
@ray.remote

def test_lambda(lamb):
  # aplicando a regressão
  ridge = sklm.Ridge(alpha = lamb)
  ridge.fit(x_tr, y_tr)             # Fit a ridge regression on the training data

  scores = skms.cross_validate(ridge, x_tr, y_tr, cv=10,
                               scoring=('r2', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error'))

  mae = np.mean(scores['test_neg_mean_absolute_error']*(-1))
  mape = np.mean(scores['test_neg_mean_absolute_percentage_error']*(-1))
  r2 = np.mean(scores['test_r2'])

  return [mae, mape, r2]

In [ ]:
result_values = ray.get([test_lambda.remote(i) for i in lambdas])
scores_MAE = [result[0] for result in result_values]
scores_MAPE = [result[1] for result in result_values]
scores_R2 = [result[2] for result in result_values]

In [ ]:
best_index = scores_MAE.index(min(scores_MAE))

print("Lambda:" + str(lambdas[best_index]))
print("Best MAE: " + str(scores_MAE[best_index]))
print("Best MAPE: " + str(scores_MAPE[best_index]))
print("R2: " + str(scores_R2[best_index]))

ridge = sklm.Ridge(alpha = lambdas[best_index])
ridge.fit(x_tr, y_tr)
plot_results(ridge.predict(x_te), y_te)

In [ ]:
y_pred = ridge.predict(x_te)
print('-----------Teste-------------')
print("MAE: " + str(mean_absolute_error(y_te, y_pred)))
print("MAPE: " + str(mean_absolute_percentage_error(y_te, y_pred)))
print("R2: " + str(r2_score(y_te, y_pred)))

##3.3 Métodos não lineares

###3.3.1. SVM

In [ ]:
from sklearn import svm

In [ ]:
svm_regressor = svm.SVR()
svm_regressor.fit(x_tr, y_tr)

**Teste com 5-Folds**

In [ ]:
train_test_model(svm_regressor, x_tr, y_tr, x_te, y_te, folds=5)

**Teste com 10-Folds**

In [ ]:
train_test_model(svm_regressor, x_tr, y_tr, x_te, y_te, folds=10)

###3.3.2. Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor as rf

In [ ]:
rf_regressor = rf(n_estimators=1000, random_state=1)
rf_regressor.fit(x_tr, y_tr)

**Teste com 5-Folds**

In [ ]:
train_test_model(rf_regressor, x_tr, y_tr, x_te, y_te, folds=5)

**Teste com 10-Folds**

In [ ]:
train_test_model(rf_regressor, x_tr, y_tr, x_te, y_te, folds=10)

###3.3.3. XGBoost

**Uma brever explicação de como treinaremos este modelo (o mesmo vale para o *multilayer perceptron* - MLP):**



*   Utilizaremos um algoritmo de otimização (otimização Bayesiana) para otimizar os hiperparâmetros dos modelos. Basicamente, hiperparâmetros são valores configuráveis que podem alterar a forma como o modelo aprende, por exemplo, em uma rede neural multicamadas (MLP) os hiperparâmetros podem ser o número de camadas, o número de nurônios em cada camada, o número de amostras passadas por vez para o modelo durante o treinamento, etc

*   Salvaremos os melhores hiperparâmetros obtidos e o melhor modelo obtido na etapa de cross validation.



In [ ]:
best = np.inf

In [ ]:
def r2_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    return 'r2', r2_score(y_true, y_pred)

In [ ]:
def cross_validation_xgb(x_train, y_train, params, folds=5):

  global best

  kf = KFold(n_splits=folds, random_state=1, shuffle=True)
  x_tr = np.array(x_train)
  y_tr = np.array(y_train)
  histories = []
  models = []
  results = []

  for train_index, test_index in kf.split(x_tr):
    xtr, xva = x_tr[train_index], x_tr[test_index]
    ytr, yva = y_tr[train_index], y_tr[test_index]

    dtrain = xgboost.DMatrix(xtr, ytr)
    dval = xgboost.DMatrix(xva, yva)

    history = {}
    model = xgboost.train(params=params, dtrain=dtrain, num_boost_round=800,
                          evals=[(dtrain, 'train'), (dval, 'val')],
                          early_stopping_rounds=15, evals_result=history,
                          verbose_eval=False)

    r2 = r2_score(yva, model.predict(dval))
    mae = mean_absolute_error(yva, model.predict(dval))
    mape = mean_absolute_percentage_error(yva, model.predict(dval))
    result = {'r2':r2, 'mae':mae, 'mape':mape}

    histories.append(history)
    models.append(model)
    results.append(result)

    if mae < best:
      best = mae
      model.save_model("best_xgb.model")

    del model

  return histories, models, results

In [ ]:
def model_optimize(learning_rate, max_depth, min_c_weight, gamma, subsample,
                   colsample_bytree):

  max_depth = round(max_depth)

  params = {'max_depth':max_depth, 'min_child_weight':min_c_weight, 'gamma':gamma,
            'subsample':subsample, 'colsample_bytree':colsample_bytree,
            'learning_rate':learning_rate, 'custom_metric':r2_eval,
            'eval_metric':["mae", "mape", "logloss"]}


  histories, models, results = cross_validation_xgb(x_tr, y_tr, params, folds=5)

  # pdb.set_trace()

  # model = load_model('_pesos_lstm1.h5')
  # os.remove("_pesos_lstm1.h5")

  plt.rcParams['figure.figsize']=(20,5)
  plt.rcParams.update({'font.size': 20})

  history_mae = np.array([h['train']['mae'] for h in histories])
  history_vmae = np.array([h['val']['mae'] for h in histories])

  plt.plot(np.mean(history_mae, axis=0))
  plt.plot(np.mean(history_vmae, axis=0))
  plt.title('model mae')
  plt.ylabel('mae')
  plt.xlabel('epoch')
  plt.legend(['train', 'validation'], loc='upper left')
  plt.show()

  gc.collect()
  scores = np.array([r['r2'] for r in results])
  return scores.mean()

In [ ]:
pbounds = {'max_depth': (3, 100),
           'learning_rate': (0.001, 1),
           'min_c_weight': (1, 100),
           'gamma': (0, 0.3),
           'subsample': (0.1, 1),
           'colsample_bytree': (0.1, 1),
        }

optimizer = BayesianOptimization(
    f=model_optimize,
    pbounds=pbounds,
    random_state=1
)

# load_logs(optimizer, logs=["./logs.json"])
# logger = JSONLogger(path="./logs.json")
# optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)
optimizer.maximize(init_points=50, n_iter=250)

**Teste de um modelo "configurado" com os melhores hiperparâmetros obtidos**

In [ ]:
best_hpp = optimizer.max['params']
best_hpp['max_depth'] = round(best_hpp['max_depth'])
best_hpp['eval_metric'] = ["mae", "mape", "logloss"]

In [ ]:
best_xgb = XGBRegressor(n_estimators=100, **best_hpp)
best_xgb.fit(x_tr, y_tr, verbose=False)

In [ ]:
y_pred = best_xgb.predict(x_te)

In [ ]:
print("MAE: " + str(mean_absolute_error(y_te, y_pred)))
print("MAPE: " + str(mean_absolute_percentage_error(y_te, y_pred)))
print("R2: " + str(r2_score(y_te, y_pred)))

**Teste do melhor modelo obtido na cross validation**

In [ ]:
best_xgb = XGBRegressor()
best_xgb.load_model('best_xgb.model')

y_pred = best_xgb.predict(x_te)

print("MAE: " + str(mean_absolute_error(y_te, y_pred)))
print("MAPE: " + str(mean_absolute_percentage_error(y_te, y_pred)))
print("R2: " + str(r2_score(y_te, y_pred)))

###3.3.4 MLP

In [ ]:
best = np.inf

In [ ]:
x_tr.shape

In [ ]:
import tensorflow
from keras.layers import Dense, Dropout
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adadelta, Adamax, RMSprop
from tensorflow.keras.layers import PReLU

In [ ]:
def cross_validation(x_train, y_train, dense_layers, neurons_1, neurons_2, neurons_3, optimizer,
                     learning_rate, dropout, folds=5, batch_size=32, callbacks=None):

  global best

  kf = KFold(n_splits=folds, random_state=1, shuffle=True)
  x_tr = np.array(x_train)
  y_tr = np.array(y_train)
  histories = []
  models = []
  results = []

  for train_index, test_index in kf.split(x_tr):
    xtr, xva = x_tr[train_index], x_tr[test_index]
    ytr, yva = y_tr[train_index], y_tr[test_index]

    model = ann(neurons_1, neurons_2, neurons_3, dense_layers, dropout, optimizer,
                learning_rate, xtr)

    history = model.fit(xtr, ytr, validation_data = (xva, yva),
                epochs=800, batch_size = batch_size, callbacks=callbacks,
                verbose=0)

    r2 = r2_score(yva, model.predict(xva))
    mae = mean_absolute_error(yva, model.predict(xva))
    mape = mean_absolute_percentage_error(yva, model.predict(xva))
    result = {'r2':r2, 'mae':mae, 'mape':mape}

    histories.append(history)
    models.append(model)
    results.append(result)

    if mae < best:
      best = mae
      model.save("best_ann.keras")

  return histories, models, results

In [ ]:
def ann(neurons_1, neurons_2, neurons_3, dense_layers, dropout, optimizer, learning_rate, xtr):

  neurons = [neurons_1, neurons_2, neurons_3]

  ann = Sequential()

  for i in range(0, dense_layers):
    if i == 0:
      ann.add(Dense(neurons[i], activation="relu", input_shape=(None, xtr.shape[1])))
      ann.add(Dropout(dropout))
    else:
      ann.add(Dense(neurons[i], activation="relu"))
      ann.add(Dropout(dropout))

  ann.add(Dense(1, activation=None))
  ann.compile(optimizer = optimizer(learning_rate=learning_rate), loss='mse', metrics = ['mae', tensorflow.keras.metrics.R2Score()])
  return ann

In [ ]:
def evaluate_network(dense_layers, neurons_1, neurons_2, neurons_3, optimizer, batch_size, learning_rate, dropout):

    dense_layers = round(dense_layers)
    neurons_1 = round(neurons_1); neurons_2 = round(neurons_2); neurons_3 = round(neurons_3)
    optimizer = round(optimizer)
    batch_size = 2**round(batch_size)

    optimizer_array = [Adam, Adadelta, Adamax, RMSprop]
    optimizer_val = optimizer_array[optimizer]

    es = EarlyStopping(monitor="val_mae", mode='min', verbose=0, patience=15)
    # checkpoint = ModelCheckpoint('_pesos_mlp.h5', monitor="val_mae", verbose=0,
    #                                   save_best_only=True, mode='min')

    histories, models, results = cross_validation(x_tr, y_tr,dense_layers, neurons_1,
                                         neurons_2, neurons_3, optimizer_val,
                                         learning_rate, dropout,folds=5,
                                         batch_size=batch_size, callbacks=es)

    plt.rcParams['figure.figsize']=(20,5)
    plt.rcParams.update({'font.size': 20})

    history_mae = np.array([h.history['mae'] for h in histories])
    history_vmae = np.array([h.history['val_mae'] for h in histories])

    plt.plot(np.mean(history_mae, axis=0))
    plt.plot(np.mean(history_vmae, axis=0))
    plt.title('model mae')
    plt.ylabel('mae')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

    #calculating score
    score = np.mean([r['r2'] for r in results])

    gc.collect()
    return score

In [ ]:
pbounds = {'dense_layers': (1, 3),
           'neurons_1': (1, 16),
           'neurons_2': (1, 8),
           'neurons_3': (1, 4),
           'optimizer': (0, 3),
           'batch_size': (5, 7),
           'learning_rate': (0.0001, 1),
           'dropout': (0, 0.5)
        }

optimizer = BayesianOptimization(
    f=evaluate_network,
    pbounds=pbounds,
    random_state=2
)

# load_logs(optimizer, logs=["./logs.json"])
# logger = JSONLogger(path="./logs.json")
# optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)
optimizer.maximize(init_points=50, n_iter=250)

**Teste de um modelo "configurado" com os melhores hiperparâmetros obtidos**

In [ ]:
params = optimizer.max['params']

In [ ]:
params['dense_layers'] = round(params['dense_layers'])
params['neurons_1'] = round(params['neurons_1'])
params['neurons_2'] = round(params['neurons_2'])
params['neurons_3'] = round(params['neurons_3'])
params['batch_size'] = 2**round(params['batch_size'])

optimizer_array = [Adam, Adadelta, Adamax, RMSprop]
params['optimizer'] = optimizer_array[round(params['optimizer'])]

In [ ]:
model = ann(params['neurons_1'], params['neurons_2'], params['neurons_3'],
            params['dense_layers'], params['dropout'], params['optimizer'],
            params['learning_rate'], x_tr)

In [ ]:
es = EarlyStopping(monitor="mae", mode='min', verbose=0, patience=15)
model.fit(x_tr, y_tr, epochs=800, batch_size = params['batch_size'], callbacks=es, verbose=0)

In [ ]:
y_pred = model.predict(x_te)

print("MAE: " + str(mean_absolute_error(y_te, y_pred)))
print("MAPE: " + str(mean_absolute_percentage_error(y_te, y_pred)))
print("R2: " + str(r2_score(y_te, y_pred)))

**Teste do melhor modelo obtido na cross validation**

In [ ]:
best_ann = keras.models.load_model('best_ann.keras')

y_pred = best_ann.predict(x_te)

print("MAE: " + str(mean_absolute_error(y_te, y_pred)))
print("MAPE: " + str(mean_absolute_percentage_error(y_te, y_pred)))
print("R2: " + str(r2_score(y_te, y_pred)))

#4. Resultados

In [ ]:
MAE: 0.4152660683467405
MAPE: 0.20966133201606202
R2: -0.0248688355671991

O melhor resultado foi obtido pelo SVM, com um MAE de 0.415, MAPE de 0.21 e $R^2$ de -0.025.

Usando o valor de $\lambda = 0.05245$ obtido a partir da transformação Box-cox. Calcualndo, portanto o erro em relação ao 'P medido', temos:

\begin{gather}
      y(\lambda) = \frac{y^\lambda - 1}{\lambda}
\end{gather}

Substituindo os valores de $\lambda$ e y($\lambda$) (MAE), temos

\begin{gather}
      0.415 = \frac{P^{0.05245} - 1}{0.05245}\\ \\
      0.415 \cdot 0.05245 + 1 = P^{0.05245} \\ \\
      P = \log_{ \; 0.05245}(0.415 \cdot 0.05245 + 1)
\end{gather}      

In [ ]:
math.log(0.415*0.05245 + 1, 0.05245)

Ou seja, o resultado acima indica que nosso modelo errou, em média, por aproximadamente $-0.0073$ mg/L da quantidade real de P medido do conjunto de teste.

In [ ]:
svm_regressor.fit(x_tr, y_tr)
y_pred = svm_regressor.predict(x_te)

In [ ]:
y_tes = y_te.tolist()
for i in range(len(y_pred)):
  y_pred[i] = math.log(y_pred[i]*0.05245 + 1, 0.05245)
  y_tes[i] = math.log(y_tes[i]*0.05245 + 1, 0.05245)

Transformamos os valores do conjunto de teste e os valores preditos pelo modelo de regressão para a unidade original (mg/L), ou seja para a unidade a anterior à aplicação do Box-cox. O gráfico abaixo mostra o desempenho do modelo.

In [ ]:
plot_results(y_pred, y_tes)

#5. Conclusões



*   Como não conheço as características do problema, não sei se os valores para os erros médios obtidos são aceitáveis;

*   Mesmo não conhecendo a natureza do problema, recomendo fortemente a busca por novas variáveis. Como vimos, a informação das variáveis B2 e B3 é redundante. Além disso, essas variáveis parecem fornecer pouca informação sobre a variável P;

*   Para termos mais confiança nos resultados, seriam necessários mais dados, pois, quanto menos dados de teste, menor nossa certeza na capacidade de generalização do modelo;

*   Por fim, a sugestão de adicionar a métrica B3/B2 parece ter melhorado os resultados.






In [ ]:
%%capture
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/CNN_Chagas/fosforo/Analise_fosforo.ipynb ./

In [ ]:
!jupyter nbconvert --to PDF "Analise_fosforo.ipynb"